# surplus 2.x.y playground notebook

wrangling with environments for devbox users using codium/vs code:

```text
$ devbox shell                    # enter devbox env
(surplus-py3.11) (devbox) $ exit  # leave poetry env
(devbox) $ codium .               # open ide
```

In [1]:
# converting nominatim keys to OUTPUT_LINE_X_KEYS format

keys = """
region, county, state, state_district, country, continent
"""

split_keys = [f'"{key.strip()}"' for key in keys.strip().split(",")]

print(f"OUTPUT_LINE_X_KEYS: Final[tuple[str, ...]] = ({','.join(split_keys)},)")

OUTPUT_LINE_X_KEYS: Final[tuple[str, ...]] = ("region","county","state","state_district","country","continent",)


In [2]:
from surplus import PlusCodeQuery, LocalCodeQuery, LatlongQuery, StringQuery
from surplus import Latlong, Result
from surplus import SurplusDefaultGeocoding

geocoding = SurplusDefaultGeocoding()

## Generic Result NamedTuple

In [3]:
nom_result = Result[int](3)

try:
    1 / 0
except Exception as exc:
    exc_result = Result[int](-1, error=exc)

print("{}\t{:<40}\t{}".format(bool(nom_result), repr(nom_result.error), nom_result.get()))
print(
    "{}\t{:<40}\t{}".format(
        bool(exc_result), repr(exc_result.error), exc_result.cry(string=True)
    )
)
print("{}\t{:<40}\t{}".format(bool(exc_result), repr(exc_result.error), exc_result.get()))

True	None                                    	3
False	ZeroDivisionError('division by zero')   	division by zero (ZeroDivisionError)


ZeroDivisionError: division by zero

## Query Types

In [ ]:
PlusCodeQuery(code="6PH58QMF+FV").to_lat_long_coord(geocoder=geocoding.geocoder)

Result(value=Latlong(latitude=1.3336875, longitude=103.7746875), error=None)

In [ ]:
plus_code = LocalCodeQuery(code="8QMF+FV", locality="Singapore").to_full_plus_code(
    geocoder=geocoding.geocoder
)

PlusCodeQuery(code=plus_code.get()).to_lat_long_coord(geocoder=geocoding.geocoder)

Result(value=Latlong(latitude=1.3336875, longitude=103.7746875), error=None)

In [ ]:
LocalCodeQuery(code="8QMF+FV", locality="Singapore").to_lat_long_coord(
    geocoder=geocoding.geocoder
)

Result(value=Latlong(latitude=1.3336875, longitude=103.7746875), error=None)

In [ ]:
LatlongQuery(
    latlong=Latlong(latitude=1.33318835, longitude=103.77461234638255)
).to_lat_long_coord(geocoder=geocoding.geocoder)

Result(value=Latlong(latitude=1.33318835, longitude=103.77461234638255), error=None)

In [ ]:
StringQuery(query="Ngee Ann Polytechnic").to_lat_long_coord(geocoder=geocoding.geocoder)

Result(value=Latlong(latitude=1.33318835, longitude=103.77461234638255), error=None)

## return dictionary of `reverser` function

all the necessary keys (see `SHAREABLE_TEXT_LINE_*` constants) should be at the top-level of the dictionary.
these keys will be casted into strings for safety guarantee when shareable text lines are being generated.

while not necessary, consider keeping the original response dict under the "raw" key.
helps with debugging using `-d/--debug`!

In [ ]:
reverser_return = {
    "amenity": "Ngee Ann Polytechnic",
    "house_number": "535",
    "road": "Clementi Road",
    "suburb": "Bukit Timah",
    "city": "Singapore",
    "county": "Northwest",
    "ISO3166-2-lvl6": "SG-03",
    "postcode": "599489",
    "country": "Singapore",
    "country_code": "sg",
    "raw": {
        "place_id": 297946059,
        "licence": "Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright",
        "osm_type": "relation",
        "osm_id": 2535118,
        "lat": "1.33318835",
        "lon": "103.77461234638255",
        "class": "amenity",
        "type": "university",
        "place_rank": 30,
        "importance": 0.34662169301918117,
        "addresstype": "amenity",
        "name": "Ngee Ann Polytechnic",
        "display_name": "Ngee Ann Polytechnic, 535, Clementi Road, Bukit Timah, Singapore, Northwest, 599489, Singapore",
        "address": {
            "amenity": "Ngee Ann Polytechnic",
            "house_number": "535",
            "road": "Clementi Road",
            "suburb": "Bukit Timah",
            "city": "Singapore",
            "county": "Northwest",
            "ISO3166-2-lvl6": "SG-03",
            "postcode": "599489",
            "country": "Singapore",
            "country_code": "sg",
        },
        "boundingbox": ["1.3289692", "1.3372184", "103.7701481", "103.7783945"],
    },
    "latitude": 1.33318835,
    "longitude": 103.77461234638255,
}

In [ ]:
import pprint

latlong = LocalCodeQuery(code="8QMF+FV", locality="Singapore").to_lat_long_coord(
    geocoder=geocoding.geocoder
)
if not latlong:
    latlong.cry()

else:
    location = geocoding.reverser(latlong.get())
    pprint.pprint(location)

{'ISO3166-2-lvl6': 'SG-03',
 'amenity': 'Ngee Ann Polytechnic',
 'city': 'Singapore',
 'country': 'Singapore',
 'country_code': 'sg',
 'county': 'Northwest',
 'house_number': '535',
 'latitude': 1.33318835,
 'longitude': 103.77461234638255,
 'neighbourhood': 'Ewart Park',
 'postcode': '599489',
 'raw': {'address': {'ISO3166-2-lvl6': 'SG-03',
                     'amenity': 'Ngee Ann Polytechnic',
                     'city': 'Singapore',
                     'country': 'Singapore',
                     'country_code': 'sg',
                     'county': 'Northwest',
                     'house_number': '535',
                     'neighbourhood': 'Ewart Park',
                     'postcode': '599489',
                     'road': 'Clementi Road',
                     'suburb': 'Bukit Timah'},
         'addresstype': 'amenity',
         'boundingbox': ['1.3289692',
                         '1.3372184',
                         '103.7701481',
                         '103.7783945'],
  

## 2.1.0: the adventure of shortening global/full Plus Codes

### testing rate-limited default geocoding functions

In [5]:
from surplus import SurplusGeocoderProtocol, SurplusReverserProtocol


test_geocoding = SurplusDefaultGeocoding(user_agent="surplus/playground")

print(location := test_geocoding.geocoder("Ngee Ann Polytechnic"))

print(reversed := test_geocoding.reverser(f"{location.latitude}, {location.longitude}"))

1.33318835, 103.77461234638255
{'amenity': 'Ngee Ann Polytechnic', 'house_number': '535', 'road': 'Clementi Road', 'neighbourhood': 'Ewart Park', 'suburb': 'Bukit Timah', 'city': 'Singapore', 'county': 'Northwest', 'ISO3166-2-lvl6': 'SG-03', 'postcode': '599489', 'country': 'Singapore', 'country_code': 'sg', 'raw': {'place_id': 250910125, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 2535118, 'lat': '1.33318835', 'lon': '103.77461234638255', 'class': 'amenity', 'type': 'university', 'place_rank': 30, 'importance': 0.34662169301918117, 'addresstype': 'amenity', 'name': 'Ngee Ann Polytechnic', 'display_name': 'Ngee Ann Polytechnic, 535, Clementi Road, Ewart Park, Bukit Timah, Singapore, Northwest, 599489, Singapore', 'address': {'amenity': 'Ngee Ann Polytechnic', 'house_number': '535', 'road': 'Clementi Road', 'neighbourhood': 'Ewart Park', 'suburb': 'Bukit Timah', 'city': 'Singapore', 'county': 'Northwest', 'ISO3166

### loop for less information until a local code is made

In [ ]:
# TODO

test1 = LocalCodeQuery("9R3J+R9", "Singapore")
test2 = LocalCodeQuery("G227+XF", "St Lucia, Queensland, Australia")

level = 13

In [ ]:
(
    response := geocoding.reverser(
        test1.to_lat_long_coord(geocoding.geocoder).get(), level=level
    )
)

{'suburb': 'Bishan',
 'city': 'Singapore',
 'county': 'Central',
 'ISO3166-2-lvl6': 'SG-01',
 'country': 'Singapore',
 'country_code': 'sg',
 'raw': {'place_id': 251115282,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 795946716,
  'lat': '1.3519117',
  'lon': '103.8489708',
  'class': 'place',
  'type': 'suburb',
  'place_rank': 19,
  'importance': 0.39184907371668787,
  'addresstype': 'suburb',
  'name': 'Bishan',
  'display_name': 'Bishan, Singapore, Central, Singapore',
  'address': {'suburb': 'Bishan',
   'city': 'Singapore',
   'county': 'Central',
   'ISO3166-2-lvl6': 'SG-01',
   'country': 'Singapore',
   'country_code': 'sg'},
  'boundingbox': ['1.3416846', '1.3679829', '103.8184512', '103.8604083']},
 'latitude': 1.3519117,
 'longitude': 103.8489708}

In [ ]:
(
    response := geocoding.reverser(
        test2.to_lat_long_coord(geocoding.geocoder).get(), level=level
    )
)

{'suburb': 'St Lucia',
 'city_district': 'St Lucia',
 'city': 'Brisbane City',
 'state': 'Queensland',
 'ISO3166-2-lvl4': 'AU-QLD',
 'postcode': '4072',
 'country': 'Australia',
 'country_code': 'au',
 'raw': {'place_id': 54477898,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'osm_type': 'node',
  'osm_id': 88800268,
  'lat': '-27.4987362',
  'lon': '153.0081642',
  'class': 'place',
  'type': 'suburb',
  'place_rank': 19,
  'importance': 0.27501,
  'addresstype': 'suburb',
  'name': 'St Lucia',
  'display_name': 'St Lucia, Brisbane City, Queensland, 4072, Australia',
  'address': {'suburb': 'St Lucia',
   'city_district': 'St Lucia',
   'city': 'Brisbane City',
   'state': 'Queensland',
   'ISO3166-2-lvl4': 'AU-QLD',
   'postcode': '4072',
   'country': 'Australia',
   'country_code': 'au'},
  'boundingbox': ['-27.5187362', '-27.4787362', '152.9881642', '153.0281642']},
 'latitude': -27.4987362,
 'longitude': 153.0081642}

## machine fingerprinting attempt

because of nominatim's acceptable usage policy  
<https://operations.osmfoundation.org/policies/nominatim/>

In [ ]:
from hashlib import shake_256 as _hashlib_shake_256
from platform import platform as _platform_platform
from socket import gethostname as _socket_gethostname
from uuid import getnode as _uuid_getnode
from surplus import VERSION, VERSION_SUFFIX


def generate_fingerprinted_user_agent() -> Result[str]:
    """
    function that attempts to return a unique user agent string.

    returns Result[str]
        this result will always have a valid value as erroneous results will have a
        resulting value of 'surplus/<version>/generic-user'
        valid results will have a value of 'surplus/<version>/<fingerprint>', where
        fingerprint is a 12 character hexadecimal string
    """
    version: str = ".".join([str(v) for v in VERSION]) + VERSION_SUFFIX

    try:
        system_info: str = _platform_platform()
        hostname: str = _socket_gethostname()
        mac_address: str = ":".join(
            [
                "{:02x}".format((_uuid_getnode() >> elements) & 0xFF)
                for elements in range(0, 2 * 6, 2)
            ][::-1]
        )
        unique_info: str = f"{version}-{system_info}-{hostname}-{mac_address}"

        print(f"{version=}")
        print(f"{system_info=}")
        print(f"{hostname=}")
        print(f"{mac_address=}")

    except Exception as exc:
        return Result[str](f"surplus/{version} (generic-user)", error=exc)

    fingerprint: str = _hashlib_shake_256(unique_info.encode()).hexdigest(5)

    return Result[str](f"surplus/{version} ({fingerprint})")